In [ ]:
import numpy as np
import pandas as pd

: 

In [ ]:
df=pd.read_csv('./Fatigue_data.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.NT

In [ ]:
df[['CT','DT']]

In [ ]:
df['new']=np.random.rand(437)

In [ ]:
df

In [ ]:
del df['new']

In [ ]:
df

In [ ]:
del df['Sl. No.']
df

In [ ]:
df.index

In [ ]:
df['RedRatio']/100.0

In [ ]:
df['RedRatio']>500

In [ ]:
df['Fatigue'].describe()

In [ ]:
df.describe()

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby('DT').Fatigue.mean()

In [ ]:
df.Fatigue.values

# Plotting

In [ ]:
import matplotlib.pyplot as mplot

In [ ]:
y = df['Fatigue']
x1 = df['CT']
x2 = df['dA']

In [ ]:
mplot.ylabel('Fatigue Strength',fontsize=10)
mplot.xlabel('Area Proportion of Inclusions Deformed by Plastic Work',fontsize=10)
mplot.plot(x2,y)

In [ ]:
mplot.scatter(x2,y)

In [ ]:
mplot.hist(df['Fatigue'],bins=15)

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(df['dA'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

features = df.columns

# Plotting histograms for all feature columns
plt.figure(figsize=(10, 25))
for i, column in enumerate(features):
    ax = plt.subplot(len(features) // 4 + 1, 4, i + 1)
    sns.histplot(data=df, x=column, kde=True)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the correlation matrix
corr = df.corr()

# Draw the heatmap of correlation matrix
f, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(corr, annot=True, ax=ax, cmap='coolwarm', fmt=".1f", 
            linewidths=.5, cbar_kws={"shrink": .5})
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.yticks(rotation=0, fontsize=14)
plt.title('Correlation Matrix', fontsize=16)
plt.show()

### t-SNE (Distributed Stochastic Neighbor Embedding)

-Distributed stochastic neighbor embedding (t-SNE) is a statistical method for visualizing high-dimensional data by giving each datapoint a location in a two or three-dimensional map. (Read more here: https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)

-Transforming the 25-dimensional feature space onto 2-dimension reveals that samples are in clusters, with a cluster of high fatigue strength separating far from all other clusters. Our job is basically find the set of experimental conditions that corresponds to that cluster.

-Also, within each cluster, data points also form strings, which means that these experiments are often done in series with slight modification on conditions, leading to the closeness in the high dimensional space.

In [ ]:
from sklearn.manifold import TSNE

# Separating features and labels
X = df.drop('Fatigue', axis=1)
y = df['Fatigue']

# Performing t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=40)
X_tsne = tsne.fit_transform(X)

# Creating a scatter plot
plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y)
plt.title('t-SNE visualization of Fatigue')
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.legend(title='Fatigue')
plt.show()

In [ ]:
np.random.seed(2)

n= len(df)

n_val =int(0.2 * n)
n_test =int(0.2 * n)
n_train = n - (n_val + n_test)

idx=np.arange(n)
np.random

df_shuffled=df.iloc[idx]

df_train=df_shuffled.iloc[:n_train].copy()
df_val=df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test=df_shuffled.iloc[n_train+n_val:].copy() 

y_train=df_train.Fatigue.values
y_val=df_val.Fatigue.values
y_test=df_test.Fatigue.values

del df_train['Fatigue']
del df_val['Fatigue']
del df_test['Fatigue']

In [ ]:
df.columns

### Determine Weights

In [ ]:
def train_linear_regression(X, y):
    ones=np.ones(X.shape[0])
    Xb=np.column_stack([ones,X])

    XTX=np.dot(X)
    XTX_inv= np.linalg.inv(XTX)
    w_best=np.dot(XTX_inv,np.dot(Xb.T,y))
    return w_best



In [ ]:
base=['NT', 'THT', 'THt', 'THQCr', 'CT', 'Ct', 'DT', 'Dt', 'QmT', 'TT', 'Tt',
       'TCr', 'C', 'Si', 'Mn', 'P', 'S', 'Ni', 'Cr', 'Cu', 'Mo', 'RedRatio',
       'dA', 'dB', 'dC']

In [ ]:
def prepare_X(df):
    df_num=df[base].copy()
    df_num=df_num.fillna(0)
    X=df_num.values
    return X